In [ ]:
import sys
sys.path.append("../../src/earthtext")

from osm import osm
import os
from progressbar import progressbar as pbar
import shapely as sh 
from pyproj import CRS
import numpy as np
epsg4326 = CRS.from_epsg(4326)
import geopandas as gpd
import pandas as pd
from importlib import reload
from rlxutils import Command
import rlxutils
reload(osm)

In [2]:
orig_partsdir = "/opt/data/osm/california-parts/"
dest_partsdir = "/opt/data/california-worldcover-chips/osm"
os.makedirs(dest_partsdir, exist_ok=True)
chip_ids_pbf = [i.split(".")[0] for i in os.listdir(orig_partsdir) if i.endswith(".pbf")]
chip_ids_geojson = [i.split(".")[0] for i in os.listdir(orig_partsdir) if i.endswith(".geojson")]

if len(set(chip_ids_pbf).intersection(set(chip_ids_geojson)))!=len(chip_ids_pbf)!=len(chip_ids_geojson):
    raise ValueError("missing chips in geojson o pbf")

In [3]:
# load geojson

In [4]:
orig_grid = []
for chip_id in chip_ids_geojson:
    with open(f"{orig_partsdir}/{chip_id}.geojson") as f:
        orig_grid.append([chip_id, sh.from_geojson(f.read())])

orig_grid = gpd.GeoDataFrame(pd.DataFrame(orig_grid, columns=['chip_id', 'geometry']), crs=epsg4326)

In [5]:
orig_grid.head()

,chip_id,geometry
0,000235e893b24,"POLYGON ((-121.87715 39.14450, -121.65850 39.1..."
1,0002c308d55fc,"POLYGON ((-114.92112 35.05498, -114.92112 35.2..."
2,0018e8bbce095,"POLYGON ((-120.25595 36.95832, -120.55074 36.9..."
3,003fa84955512,"POLYGON ((-119.13010 35.78528, -119.40375 35.7..."
4,004aaa9df44f2,"POLYGON ((-124.14191 40.08352, -124.24849 40.0..."


In [6]:
dest_grid = gpd.read_file("/opt/data/california-worldcover-chips.fgb")
dest_grid.head()

,col,row,geometry
0,131584,119552,"POLYGON ((-114.01333 32.01600, -114.01333 32.0..."
1,131584,119296,"POLYGON ((-114.01333 32.03733, -114.01333 32.0..."
2,131328,119296,"POLYGON ((-114.03467 32.03733, -114.03467 32.0..."
3,131328,119552,"POLYGON ((-114.03467 32.01600, -114.03467 32.0..."
4,131072,119552,"POLYGON ((-114.05600 32.01600, -114.05600 32.0..."


In [15]:
chip_ids = ['0d0d59f364f72', '26017774321e9']
_orig_grid = orig_grid[orig_grid.chip_id.isin(chip_ids)]
_orig_grid.shape

(2, 2)

In [ ]:
c = _orig_grid.geometry.values[1]

dg = dest_grid[[i.intersects(c) for i in dest_grid.geometry.values]]
pd.concat([dg, _orig_grid]).explore()

In [16]:
_orig_grid.head()

,chip_id,geometry
214,0d0d59f364f72,"POLYGON ((-122.24843 37.22020, -122.24843 37.3..."
648,26017774321e9,"POLYGON ((-121.22865 36.97216, -121.47409 36.9..."


In [19]:
notfound = []
for count, (_, orig_row) in enumerate(_orig_grid.iterrows()):
    et = rlxutils.ElapsedTimes()
    orig_row_geom = orig_row.geometry
    dg = dest_grid[[i.intersects(orig_row_geom) for i in dest_grid.geometry.values]]
    print (f"{count}/{len(orig_grid)}", orig_row.chip_id, 'dest_chips', len(dg), flush=True)

    parquets_cache = {}
    for _, dest_row in pbar(dg.iterrows(), max_value=len(dg)):
        
        dest_row_geom = dest_row.geometry
        
        dest_parquet = f"{dest_partsdir}/{osm.get_region_hash(dest_row_geom)}.parquet"
        if os.path.isfile(dest_parquet):
            continue

        with et("ogintersect"):
            og = orig_grid[[i.intersects(dest_row_geom) for i in orig_grid.geometry.values]]
        
        if len(og) == 0:
            continue
            
        dgdata = []
        for chip_id in og.chip_id.values:
            if chip_id in parquets_cache.keys():
                p = parquets_cache[chip_id]
            else:
                with et("read_parquet"):
                    fname = f"{orig_partsdir}/{chip_id}.parquet"
                    if not os.path.isfile(fname):
                        notfound.append(fname)
                        continue
                    p = gpd.read_parquet(fname)
                    parquets_cache[chip_id]=p
            
            with et("dgintersect"):
                p = p[p.intersects(dest_row_geom)]
            dgdata.append(p)
        if len(dgdata)==0:
            continue
        dgdata = pd.concat(dgdata)
        with et("save"):
            dgdata.to_parquet(dest_parquet)
    #print(et)

0/984 0d0d59f364f72 dest_chips 126


100% (126 of 126) |######################| Elapsed Time: 0:00:11 Time:  0:00:110001


1/984 26017774321e9 dest_chips 108


100% (108 of 108) |######################| Elapsed Time: 0:00:00 Time:  0:00:000000


In [10]:
len(notfound)

154